<a href="https://colab.research.google.com/github/jwengr/dacon/blob/main/%EC%8B%9C%EC%8A%A4%ED%85%9C%20%ED%92%88%EC%A7%88%20%EB%B3%80%ED%99%94%EB%A1%9C%20%EC%9D%B8%ED%95%9C%20%EC%82%AC%EC%9A%A9%EC%9E%90%20%EB%B6%88%ED%8E%B8%20%EC%98%88%EC%A7%80%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/Preprecessing_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from datetime import datetime
import os
import scipy
from scipy import stats

In [2]:
pd.options.mode.chained_assignment = None

In [3]:
defaultpath = 'drive/My Drive/dacon/lg'

err test err

In [4]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
err_df.dropna(axis=0, inplace=True)
err_df.drop_duplicates(inplace=True)
err_df['user_id'] = err_df['user_id'].astype(np.int32)
err_df['month']=err_df['time'].apply(lambda x:x[4:6])
err_df['day']=err_df['time'].apply(lambda x:x[6:8])
err_df['hour']=err_df['time'].apply(lambda x:x[8:10])
err_df = pd.concat([err_df,pd.get_dummies(err_df['errtype'],prefix='et')],axis=1)
err_df['all']=1


In [16]:
l = []
for key in ['_mean','_std','_skew','_sum','_kurt','_chi_s','_chi_p']:
    for col in err_df.columns[9:-1]:
        l = l+[str(col)+key]
err_arr = pd.DataFrame(np.zeros((15000,41*7+1)),columns=l+['all'])
for i in tqdm(range(10000,25000,1)):
    temp = err_df[err_df['user_id']==i]
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_et = temp.groupby(['month','day','hour']).sum().drop('user_id',axis=1).reset_index().drop(['month','day','hour','all'],axis=1)
    mean = temp_et.mean().values
    std = temp_et.std().values
    skew = temp_et.skew()
    sum = temp_et.sum()
    kurt = temp_et.kurt()
    s,p =  stats.chisquare(temp_et,axis=1)
    
    err_arr.loc[i-10000,:] = np.hstack([mean,std,skew,sum,kurt,s,p])
    break
# problem_df= pd.read_csv(defaultpath+'/dataset/train_problem_data.csv',encoding='utf-8',dtype=np.str)
# problem_ppdf = pd.merge(problem_df.astype(np.int32).drop_duplicates(),pd.DataFrame(range(10000,25000),columns=['user_id']),how='outer',on='user_id')
# problem_ppdf['time'][problem_ppdf['time'].notna()] = 1
# problem_ppdf.fillna(0,inplace=True)
# problem_ppdf = problem_ppdf.astype(np.int32)
# problem_ppdf.sort_values('user_id',inplace=True)
# problem_ppdf.drop_duplicates(inplace=True)
# err = pd.concat([problem_ppdf.reset_index(drop=True),err_arr],axis=1)
# err.fillna(0,inplace=True)
# err.to_pickle(defaultpath+'/dataset/pp_err_et.pkl')

AttributeError: ignored

In [13]:
temp_et

,et_1,et_10,et_11,et_12,et_13,et_14,et_15,et_16,et_17,et_18,et_19,et_2,et_20,et_21,et_22,et_23,et_24,et_25,et_26,et_27,et_28,et_3,et_30,et_31,et_32,et_33,et_34,et_35,et_36,et_37,et_38,et_39,et_4,et_40,et_41,et_42,et_5,et_6,et_7,et_8,et_9
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
chi[0].shape

(716,)

In [ ]:
err.head()

,user_id,time,et_1_mean,et_10_mean,et_11_mean,et_12_mean,et_13_mean,et_14_mean,et_15_mean,et_16_mean,et_17_mean,et_18_mean,et_19_mean,et_2_mean,et_20_mean,et_21_mean,et_22_mean,et_23_mean,et_24_mean,et_25_mean,et_26_mean,et_27_mean,et_28_mean,et_3_mean,et_30_mean,et_31_mean,et_32_mean,et_33_mean,et_34_mean,et_35_mean,et_36_mean,et_37_mean,et_38_mean,et_39_mean,et_4_mean,et_40_mean,et_41_mean,et_42_mean,et_5_mean,et_6_mean,...,et_10_kurt,et_11_kurt,et_12_kurt,et_13_kurt,et_14_kurt,et_15_kurt,et_16_kurt,et_17_kurt,et_18_kurt,et_19_kurt,et_2_kurt,et_20_kurt,et_21_kurt,et_22_kurt,et_23_kurt,et_24_kurt,et_25_kurt,et_26_kurt,et_27_kurt,et_28_kurt,et_3_kurt,et_30_kurt,et_31_kurt,et_32_kurt,et_33_kurt,et_34_kurt,et_35_kurt,et_36_kurt,et_37_kurt,et_38_kurt,et_39_kurt,et_4_kurt,et_40_kurt,et_41_kurt,et_42_kurt,et_5_kurt,et_6_kurt,et_7_kurt,et_8_kurt,et_9_kurt
0,10000,0,0.0,0.009777,0.020950,0.022346,0.001397,0.013966,0.082402,0.083799,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.044693,0.001397,0.001397,0.011173,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.145251,0.000000,0.000000,0.000000,0.000000,0.001397,...,97.986885,43.063233,40.060430,716.00000,243.833180,11.080271,20.330191,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,17.552486,716.000000,716.000000,85.112870,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,11.508521,0.000000,0.000000,0.000000,0.000000,716.00000,716.000000,0.0,0.0
1,10001,1,0.0,0.000000,0.013947,0.015342,0.001395,0.011158,0.199442,0.178522,0.000000,0.005579,0.001395,0.0,0.004184,0.001395,1.054393,1.047420,0.006974,0.001395,0.030683,0.000000,0.000000,0.000000,0.0,0.348675,0.0,0.013947,0.025105,0.0,0.001395,0.001395,0.000000,0.0,0.000000,0.157601,0.078103,0.001395,0.066946,0.001395,...,0.000000,67.190137,102.128778,717.00000,175.711836,9.612965,13.963380,0.0,355.988799,717.0,0.0,486.082035,717.0,82.715768,82.836970,275.230101,717.0,60.118263,0.000000,0.000000,0.000000,0.0,7.741292,0.0,67.190137,176.895725,0.0,717.0,717.0,0.000000,0.0,0.000000,168.012216,43.208709,717.000000,422.266785,717.00000,717.000000,0.0,0.0
2,10002,0,0.0,0.001399,0.018182,0.019580,0.001399,0.005594,0.072727,0.072727,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.002797,0.000000,0.034965,0.002797,0.002797,0.002797,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.183217,0.000000,0.000000,0.000000,0.001399,0.002797,...,715.000000,50.378507,46.423787,715.00000,585.541252,8.898920,13.649338,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,715.000000,0.0,23.810795,354.988768,354.988768,354.988768,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,7.780490,0.000000,0.000000,0.000000,715.000000,715.00000,715.000000,0.0,0.0
3,10003,0,0.0,0.000000,0.012894,0.012894,0.000000,0.000000,0.064470,0.042980,0.001433,0.000000,0.000000,0.0,0.000000,0.000000,0.040115,0.027221,0.000000,0.000000,0.081662,0.000000,0.000000,0.000000,0.0,0.093123,0.0,0.011461,0.000000,0.0,0.001433,0.001433,0.002865,0.0,0.000000,0.024355,0.001433,0.000000,0.002865,0.001433,...,0.000000,73.099790,73.099790,0.00000,0.000000,55.042319,18.452074,698.0,0.000000,0.0,0.0,0.000000,0.0,33.772722,57.801587,0.000000,0.0,20.198724,0.000000,0.000000,0.000000,0.0,26.052601,0.0,82.862560,0.000000,0.0,698.0,698.0,346.488493,0.0,0.000000,67.074994,698.000000,0.000000,346.488493,698.00000,698.000000,0.0,0.0
4,10004,1,0.0,0.000000,0.022284,0.026462,0.004178,0.006964,0.199164,0.126741,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.194986,0.165738,0.000000,0.000000,0.045961,0.000000,0.000000,0.000000,0.0,0.245125,0.0,0.022284,0.000000,0.0,0.001393,0.001393,0.000000,0.0,0.001393,0.005571,0.000000,0.002786,0.000000,0.004178,...,0.000000,62.235881,49.374192,235.98603,139.585508,29.976928,68.644820,0.0,0.000000,0.0,0.0,0.000000,0.0,359.306725,397.723527,0.000000,0.0,37.090951,0.000000,0.000000,0.000000,0.0,233.193855,0.0,62.235881,0.000000,0.0,718.0,7

In [ ]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df.dropna(axis=0, inplace=True)
test_err_df.drop_duplicates(inplace=True)
test_err_df['user_id'] = test_err_df['user_id'].astype(np.int32)
test_err_df['month']=test_err_df['time'].apply(lambda x:x[4:6])
test_err_df['day']=test_err_df['time'].apply(lambda x:x[6:8])
test_err_df['hour']=test_err_df['time'].apply(lambda x:x[8:10])
test_err_df = pd.concat([test_err_df,pd.get_dummies(test_err_df['errtype'],prefix='et')],axis=1)
l = []
for key in ['_mean','_std','_skew','_sum','_kurt']:
    for col in test_err_df.columns[9:]:
        l = l+[str(col)+key]
err_arr = pd.DataFrame(np.zeros((14999,41*5)),columns=l)
for i in tqdm(range(30000,44999,1)):
    temp = test_err_df[test_err_df['user_id']==i]
    if len(temp) ==0: continue
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_et = temp.groupby(['month','day','hour']).sum().drop('user_id',axis=1).reset_index().drop(['month','day','hour'],axis=1)
    mean = temp_et.mean().values
    std = temp_et.std().values
    skew = temp_et.skew()
    sum = temp_et.sum()
    kurt = temp_et.kurt()
    err_arr.loc[i-30000,:] = np.hstack([mean,std,skew,sum,kurt])
err_arr.fillna(0,inplace=True)
err_arr.to_pickle(defaultpath+'/dataset/pp_test_err_et.pkl')

In [ ]:
err_arr.head()

,et_1_mean,et_10_mean,et_11_mean,et_12_mean,et_13_mean,et_14_mean,et_15_mean,et_16_mean,et_17_mean,et_18_mean,et_19_mean,et_2_mean,et_20_mean,et_21_mean,et_22_mean,et_23_mean,et_24_mean,et_25_mean,et_26_mean,et_27_mean,et_28_mean,et_3_mean,et_30_mean,et_31_mean,et_32_mean,et_33_mean,et_34_mean,et_35_mean,et_36_mean,et_37_mean,et_38_mean,et_39_mean,et_4_mean,et_40_mean,et_41_mean,et_42_mean,et_5_mean,et_6_mean,et_7_mean,et_8_mean,...,et_10_kurt,et_11_kurt,et_12_kurt,et_13_kurt,et_14_kurt,et_15_kurt,et_16_kurt,et_17_kurt,et_18_kurt,et_19_kurt,et_2_kurt,et_20_kurt,et_21_kurt,et_22_kurt,et_23_kurt,et_24_kurt,et_25_kurt,et_26_kurt,et_27_kurt,et_28_kurt,et_3_kurt,et_30_kurt,et_31_kurt,et_32_kurt,et_33_kurt,et_34_kurt,et_35_kurt,et_36_kurt,et_37_kurt,et_38_kurt,et_39_kurt,et_4_kurt,et_40_kurt,et_41_kurt,et_42_kurt,et_5_kurt,et_6_kurt,et_7_kurt,et_8_kurt,et_9_kurt
0,0.0,0.0,0.022440,0.022440,0.000000,0.004208,0.092567,0.051893,0.012623,0.004208,0.0,0.0,0.001403,0.0,1.378682,1.834502,0.007013,0.002805,0.044881,0.000000,0.000000,0.000000,0.0,0.116410,0.002805,0.022440,0.000000,0.001403,0.004208,0.004208,0.000000,0.002805,0.000000,0.129032,0.000000,0.002805,0.061711,0.001403,0.001403,0.0,...,0.0,39.872924,39.872924,0.000000,234.319265,28.346889,81.173361,74.766692,713.0,0.0,0.0,713.0,0.0,97.251433,159.941840,633.614041,713.0,40.108001,0.000000,0.000000,0.000000,0.0,61.254975,713.000000,39.872924,0.000000,713.000000,234.319265,234.319265,0.0,713.000000,0.000000,43.865518,0.000000,353.988736,399.713106,713.000000,713.000000,0.0,0.0
1,0.0,0.0,0.016783,0.022378,0.005594,0.008392,0.074126,0.076923,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.033566,0.001399,0.001399,0.004196,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.137063,0.000000,0.000000,0.000000,0.004196,0.005594,0.005594,0.0,...,0.0,54.992594,39.997928,174.985285,220.303667,46.212144,49.112430,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,25.009327,715.000000,715.000000,234.985971,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,29.981860,0.000000,0.000000,0.000000,234.985971,174.985285,174.985285,0.0,0.0
2,0.0,0.0,0.022253,0.022253,0.000000,0.001391,0.240612,0.184979,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.083449,0.055633,0.002782,0.000000,0.055633,0.000000,0.000000,0.000000,0.0,0.319889,0.000000,0.022253,0.000000,0.000000,0.001391,0.001391,0.000000,0.019471,0.000000,0.157163,0.001391,0.002782,0.057024,0.002782,0.001391,0.0,...,0.0,40.247937,40.247937,0.000000,719.000000,12.231965,6.988999,0.000000,0.0,0.0,0.0,0.0,0.0,26.698410,38.689021,356.988831,0.0,27.228032,0.000000,0.000000,0.000000,0.0,3.488584,0.000000,40.247937,0.000000,0.000000,719.000000,719.000000,0.0,540.837853,0.000000,155.313442,719.000000,356.988831,522.769781,356.988831,719.000000,0.0,0.0
3,0.0,0.0,0.005739,0.007174,0.001435,0.010043,0.011478,0.017217,0.000000,0.001435,0.0,0.0,0.001435,0.0,0.004304,0.002869,0.000000,0.000000,0.007174,0.002869,0.002869,0.000000,0.0,0.192253,0.000000,0.005739,0.010043,0.004304,0.001435,0.001435,0.000000,0.000000,0.000000,0.057389,0.011478,0.000000,0.104735,0.002869,0.002869,0.0,...,0.0,170.484905,135.385071,697.000000,177.320806,170.711538,219.904627,0.000000,697.0,0.0,0.0,697.0,0.0,472.481863,345.988477,0.000000,0.0,281.053523,345.988477,345.988477,0.000000,0.0,19.787657,0.000000,170.484905,599.020758,472.481863,697.000000,697.000000,0.0,0.000000,0.000000,65.053334,149.768528,0.000000,569.082585,345.988477,345.988477,0.0,0.0
4,0.0,0.0,0.013889,0.016667,0.001389,0.013889,0.109722,0.093056,0.008333,0.000000,0.0,0.0,0.000000,0.0,0.013889,0.011111,0.000000,0.000000,0.019444,0.001389,0.001389,0.000000,0.0,0.461111,0.211111,0.015278,0.006944,0.001389,0.001389,0.001389,0.001389,0.000000,0.002778,0.066667,0.125000,0.000000,0.008333,0.001389,0.001389,0.0,...,0.0,116.161721,91.491849,720.000000,639.865020,31.953521,54.261611,115.819465,0.0,0.0,0.0,0.0,0.0,135.833258,154.828763

fwver test fwver

In [ ]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
err_df.dropna(axis=0, inplace=True)
err_df.drop_duplicates(inplace=True)
err_df['user_id'] = err_df['user_id'].astype(np.int32)
err_df['month']=err_df['time'].apply(lambda x:x[4:6])
err_df['day']=err_df['time'].apply(lambda x:x[6:8])
err_df['hour']=err_df['time'].apply(lambda x:x[8:10])
err_df = pd.concat([err_df,pd.get_dummies(err_df['fwver']).loc[:,list(set(test_err_df['fwver'].values)& set(err_df['fwver'].values))]],axis=1)

l = []
for key in ['_mean','_std','_skew','_sum','_kurt']:
    for col in err_df.columns[9:]:
        l = l+[str(col)+key]
err_arr = pd.DataFrame(np.zeros((14999,31*5)),columns=l)
for i in tqdm(range(10000,25000,1)):
    temp = err_df[err_df['user_id']==i]
    if len(temp) ==0: continue
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_fv = temp.groupby(['month','day','hour']).sum().drop('user_id',axis=1).reset_index().drop(['month','day','hour'],axis=1)
    mean = temp_fv.mean().values
    std = temp_fv.std().values
    skew = temp_fv.skew()
    sum = temp_fv.sum()
    kurt = temp_fv.kurt()
    err_arr.loc[i-10000,:] = np.hstack([mean,std,skew,sum,kurt])
problem_df= pd.read_csv(defaultpath+'/dataset/train_problem_data.csv',encoding='utf-8',dtype=np.str)
problem_ppdf = pd.merge(problem_df.astype(np.int32).drop_duplicates(),pd.DataFrame(range(10000,25000),columns=['user_id']),how='outer',on='user_id')
problem_ppdf['time'][problem_ppdf['time'].notna()] = 1
problem_ppdf.fillna(0,inplace=True)
problem_ppdf = problem_ppdf.astype(np.int32)
problem_ppdf.sort_values('user_id',inplace=True)
problem_ppdf.drop_duplicates(inplace=True)
err = pd.concat([problem_ppdf.reset_index(drop=True),err_arr],axis=1)
err.fillna(0,inplace=True)
err.to_pickle(defaultpath+'/dataset/pp_err_fv.pkl')

In [ ]:
err

,user_id,time,8.5.3_mean,05.15.2092_mean,04.22.1684_mean,04.33.1125_mean,05.15.2138_mean,04.16.3439_mean,04.22.1666_mean,04.73.2237_mean,04.22.1750_mean,05.66.3237_mean,03.11.1167_mean,04.73.2571_mean,04.22.1656_mean,04.82.1684_mean,03.11.1149_mean,05.66.3571_mean,04.16.3571_mean,05.15.3104_mean,04.22.1778_mean,10_mean,04.16.3569_mean,04.33.1185_mean,05.15.2120_mean,04.82.1730_mean,04.33.1149_mean,04.33.1171_mean,04.82.1778_mean,04.16.3553_mean,04.33.1261_mean,03.11.1141_mean,05.15.2114_mean,8.5.3_std,05.15.2092_std,04.22.1684_std,04.33.1125_std,05.15.2138_std,04.16.3439_std,04.22.1666_std,...,05.15.2120_sum,04.82.1730_sum,04.33.1149_sum,04.33.1171_sum,04.82.1778_sum,04.16.3553_sum,04.33.1261_sum,03.11.1141_sum,05.15.2114_sum,8.5.3_kurt,05.15.2092_kurt,04.22.1684_kurt,04.33.1125_kurt,05.15.2138_kurt,04.16.3439_kurt,04.22.1666_kurt,04.73.2237_kurt,04.22.1750_kurt,05.66.3237_kurt,03.11.1167_kurt,04.73.2571_kurt,04.22.1656_kurt,04.82.1684_kurt,03.11.1149_kurt,05.66.3571_kurt,04.16.3571_kurt,05.15.3104_kurt,04.22.1778_kurt,10_kurt,04.16.3569_kurt,04.33.1185_kurt,05.15.2120_kurt,04.82.1730_kurt,04.33.1149_kurt,04.33.1171_kurt,04.82.1778_kurt,04.16.3553_kurt,04.33.1261_kurt,03.11.1141_kurt,05.15.2114_kurt
0,10000,0,0.0,0.0,0.0,0.0,0.441341,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.206356,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.546274,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,10001,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.513250,0.0,0.0,0.0,0.0,0.0,0.0,2.757322,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1977.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,35.954801,0.0,0.0,0.0,0.0,0.0,0.0,80.447343,0.0,0.0
2,10002,0,0.0,0.0,0.0,0.0,0.426573,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.133211,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.960105,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,10003,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.114613,0.0,0.0,0.0,0.0,0.0,0.0,0.310888,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,217.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,58.065126,0.0,0.0,0.0,0.0,0.0,0.0,24.759828,0.0,0.0
4,10004,1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.898329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.183844,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,180.790861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.309110,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.805907,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,8.804419,0.0,0.0
14996,24996,0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df.dropna(axis=0, inplace=True)
test_err_df.drop_duplicates(inplace=True)
test_err_df['user_id'] = test_err_df['user_id'].astype(np.int32)
test_err_df['month']=test_err_df['time'].apply(lambda x:x[4:6])
test_err_df['day']=test_err_df['time'].apply(lambda x:x[6:8])
test_err_df['hour']=test_err_df['time'].apply(lambda x:x[8:10])
test_err_df = pd.concat([test_err_df,pd.get_dummies(test_err_df['fwver']).loc[:,list(set(test_err_df['fwver'].values)& set(err_df['fwver'].values))]],axis=1)

l = []
for key in ['_mean','_std','_skew','_sum','_kurt']:
    for col in test_err_df.columns[9:]:
        l = l+[str(col)+key]
err_arr = pd.DataFrame(np.zeros((14999,31*5)),columns=l)
for i in tqdm(range(30000,44999,1)):
    temp = test_err_df[test_err_df['user_id']==i]
    if len(temp) ==0: continue
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_fv = temp.groupby(['month','day','hour']).sum().drop('user_id',axis=1).reset_index().drop(['month','day','hour'],axis=1)
    mean = temp_fv.mean().values
    std = temp_fv.std().values
    skew = temp_fv.skew()
    sum = temp_fv.sum()
    kurt = temp_fv.kurt()
    err_arr.loc[i-30000,:] = np.hstack([mean,std,skew,sum,kurt])
err_arr.fillna(0,inplace=True)
err_arr.to_pickle(defaultpath+'/dataset/pp_test_err_fv.pkl')

In [ ]:
err_arr

,8.5.3_mean,05.15.2092_mean,04.22.1684_mean,04.33.1125_mean,05.15.2138_mean,04.16.3439_mean,04.22.1666_mean,04.73.2237_mean,04.22.1750_mean,05.66.3237_mean,03.11.1167_mean,04.73.2571_mean,04.22.1656_mean,04.82.1684_mean,03.11.1149_mean,05.66.3571_mean,04.16.3571_mean,05.15.3104_mean,04.22.1778_mean,10_mean,04.16.3569_mean,04.33.1185_mean,05.15.2120_mean,04.82.1730_mean,04.33.1149_mean,04.33.1171_mean,04.82.1778_mean,04.16.3553_mean,04.33.1261_mean,03.11.1141_mean,05.15.2114_mean,8.5.3_std,05.15.2092_std,04.22.1684_std,04.33.1125_std,05.15.2138_std,04.16.3439_std,04.22.1666_std,04.73.2237_std,04.22.1750_std,...,05.15.2120_sum,04.82.1730_sum,04.33.1149_sum,04.33.1171_sum,04.82.1778_sum,04.16.3553_sum,04.33.1261_sum,03.11.1141_sum,05.15.2114_sum,8.5.3_kurt,05.15.2092_kurt,04.22.1684_kurt,04.33.1125_kurt,05.15.2138_kurt,04.16.3439_kurt,04.22.1666_kurt,04.73.2237_kurt,04.22.1750_kurt,05.66.3237_kurt,03.11.1167_kurt,04.73.2571_kurt,04.22.1656_kurt,04.82.1684_kurt,03.11.1149_kurt,05.66.3571_kurt,04.16.3571_kurt,05.15.3104_kurt,04.22.1778_kurt,10_kurt,04.16.3569_kurt,04.33.1185_kurt,05.15.2120_kurt,04.82.1730_kurt,04.33.1149_kurt,04.33.1171_kurt,04.82.1778_kurt,04.16.3553_kurt,04.33.1261_kurt,03.11.1141_kurt,05.15.2114_kurt
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.514727,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,3.228612,0.086957,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,2302.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.809022,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,125.157517,323.384518,0.0,0.0
1,0.0,0.0,0.0,0.0,0.397203,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.42467,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.618987,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.998609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.257302,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,1.973713,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,12.425193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,142.161245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.301291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.169297,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,1.668991,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,307.107901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,263.656957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.004167,0.0,0.0,0.863889,0.341667,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,...,0.0,0.0,3.0,0.0,0.0,622.0,246.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,720.0,0.0,0.0,27.785951,107.441146,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.559441,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,1115.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.033897,0.000000,0.0,0.0
14

model test model

In [ ]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
err_df.dropna(axis=0, inplace=True)
err_df.drop_duplicates(inplace=True)
err_df['user_id'] = err_df['user_id'].astype(np.int32)
err_df['month']=err_df['time'].apply(lambda x:x[4:6])
err_df['day']=err_df['time'].apply(lambda x:x[6:8])
err_df['hour']=err_df['time'].apply(lambda x:x[8:10])
err_df = pd.concat([err_df,pd.get_dummies(err_df['model_nm'])],axis=1)
l = []
for key in ['_mean','_std','_skew','_sum','_kurt']:
    for col in err_df.columns[9:]:
        l = l+[str(col)+key]
err_arr = pd.DataFrame(np.zeros((14999,9*5)),columns=l)
for i in tqdm(range(10000,25000,1)):
    temp = err_df[err_df['user_id']==i]
    if len(temp) ==0: continue
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_m = temp.groupby(['month','day','hour']).sum().drop('user_id',axis=1).reset_index().drop(['month','day','hour'],axis=1)
    mean = temp_m.mean().values
    std = temp_m.std().values
    skew = temp_m.skew()
    sum = temp_m.sum()
    kurt = temp_m.kurt()
    err_arr.loc[i-10000,:] = np.hstack([mean,std,skew,sum,kurt])
problem_df= pd.read_csv(defaultpath+'/dataset/train_problem_data.csv',encoding='utf-8',dtype=np.str)
problem_ppdf = pd.merge(problem_df.astype(np.int32).drop_duplicates(),pd.DataFrame(range(10000,25000),columns=['user_id']),how='outer',on='user_id')
problem_ppdf['time'][problem_ppdf['time'].notna()] = 1
problem_ppdf.fillna(0,inplace=True)
problem_ppdf = problem_ppdf.astype(np.int32)
problem_ppdf.sort_values('user_id',inplace=True)
problem_ppdf.drop_duplicates(inplace=True)
err = pd.concat([problem_ppdf.reset_index(drop=True),err_arr],axis=1)
err.fillna(0,inplace=True)
err.to_pickle(defaultpath+'/dataset/pp_err_m.pkl')

In [ ]:
err

,user_id,time,model_0_mean,model_1_mean,model_2_mean,model_3_mean,model_4_mean,model_5_mean,model_6_mean,model_7_mean,model_8_mean,model_0_std,model_1_std,model_2_std,model_3_std,model_4_std,model_5_std,model_6_std,model_7_std,model_8_std,model_0_skew,model_1_skew,model_2_skew,model_3_skew,model_4_skew,model_5_skew,model_6_skew,model_7_skew,model_8_skew,model_0_sum,model_1_sum,model_2_sum,model_3_sum,model_4_sum,model_5_sum,model_6_sum,model_7_sum,model_8_sum,model_0_kurt,model_1_kurt,model_2_kurt,model_3_kurt,model_4_kurt,model_5_kurt,model_6_kurt,model_7_kurt,model_8_kurt
0,10000,0,0.000000,0.0,0.000000,0.441341,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.206356,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3.311166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,12.546274,0.0,0.0,0.0,0.0,0.0
1,10001,1,0.000000,0.0,3.270572,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,17.408320,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,8.719671,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,79.129183,0.000000,0.0,0.0,0.0,0.0,0.0
2,10002,0,0.000000,0.0,0.000000,0.426573,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.133211,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,3.683066,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,305.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,19.960105,0.0,0.0,0.0,0.0,0.0
3,10003,0,0.000000,0.0,0.425501,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.267985,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,4.175940,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,297.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,21.691113,0.000000,0.0,0.0,0.0,0.0,0.0
4,10004,1,1.082173,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,3.498585,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,10.542455,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,777.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161.686932,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,0,0.000000,0.0,0.805907,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,2.116393,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,3.000155,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,191.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,8.804419,0.000000,0.0,0.0,0.0,0.0,0.0
14996,24996,0,0.000000,0.0,0.000000,4.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
14997,24997,1,1.147265,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,2.569685,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.315854,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,818.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.444860,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
14998,24998,1,0.201964,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,1.360001,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,11.902798,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,144.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,192.453860,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [ ]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
test_err_df.dropna(axis=0, inplace=True)
test_err_df.drop_duplicates(inplace=True)
test_err_df['user_id'] = test_err_df['user_id'].astype(np.int32)
test_err_df['month']=test_err_df['time'].apply(lambda x:x[4:6])
test_err_df['day']=test_err_df['time'].apply(lambda x:x[6:8])
test_err_df['hour']=test_err_df['time'].apply(lambda x:x[8:10])
test_err_df = pd.concat([test_err_df,pd.get_dummies(test_err_df['model_nm'])],axis=1)
l = []
for key in ['_mean','_std','_skew','_sum','_kurt']:
    for col in test_err_df.columns[9:]:
        l = l+[str(col)+key]
err_arr = pd.DataFrame(np.zeros((14999,9*5)),columns=l)
for i in tqdm(range(30000,44999,1)):
    temp = test_err_df[test_err_df['user_id']==i]
    if len(temp) ==0: continue
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_m = temp.groupby(['month','day','hour']).sum().drop('user_id',axis=1).reset_index().drop(['month','day','hour'],axis=1)
    mean = temp_m.mean().values
    std = temp_m.std().values
    skew = temp_m.skew()
    sum = temp_m.sum()
    kurt = temp_m.kurt()
    err_arr.loc[i-30000,:] = np.hstack([mean,std,skew,sum,kurt])
err_arr.fillna(0,inplace=True)
err_arr.to_pickle(defaultpath+'/dataset/pp_test_err_m.pkl')

In [ ]:
quality_df

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,month,day,hour,04.22.1684,04.33.1125,04.73.2237,05.15.2138,04.16.3439,04.22.1750,05.66.3237,03.11.1167,04.73.2571,04.82.1684,03.11.1149,05.66.3571,04.16.3571,09.17.1431,04.22.1778,04.33.1185,05.15.2120,04.33.1149,04.82.1778,04.16.3553,04.33.1261,05.15.2114
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,4,0,0,11,29,09,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,20201129090000,10000,05.15.2138,0.0,0,0.0,4,0,0,0,0,4,0,0,11,29,09,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,20201130210000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,8,0,0,11,30,21,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
22,20201130210000,10000,05.15.2138,0.0,0,0.0,8,0,0,0,0,8,0,0,11,30,21,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24,20201104110000,10002,05.15.2138,0.0,0,0.0,0,0,0,0,1,0,0,0,11,04,11,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828612,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,17,0,0,11,24,03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
828615,20201124033000,24997,04.22.1778,0.0,0,0.0,2,0,0,0,0,17,0,0,11,24,03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,1,0,0,0,0,17,0,0,11,24,03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,3,0,0,0,0,17,0,0,11,24,03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


qulaity testquality

In [ ]:
l

['quality_1_mean',
 'quality_2_mean',
 'quality_5_mean',
 'quality_6_mean',
 'quality_7_mean',
 'quality_8_mean',
 'quality_9_mean',
 'quality_10_mean',
 'quality_11_mean',
 'quality_12_mean',
 '04.22.1684_mean',
 '04.33.1125_mean',
 '04.73.2237_mean',
 '05.15.2138_mean',
 '04.16.3439_mean',
 '04.22.1750_mean',
 '05.66.3237_mean',
 '03.11.1167_mean',
 '04.73.2571_mean',
 '04.82.1684_mean',
 '03.11.1149_mean',
 '05.66.3571_mean',
 '04.16.3571_mean',
 '09.17.1431_mean',
 '04.22.1778_mean',
 '04.33.1185_mean',
 '05.15.2120_mean',
 '04.33.1149_mean',
 '04.82.1778_mean',
 '04.16.3553_mean',
 '04.33.1261_mean',
 '05.15.2114_mean',
 'quality_1_std',
 'quality_2_std',
 'quality_5_std',
 'quality_6_std',
 'quality_7_std',
 'quality_8_std',
 'quality_9_std',
 'quality_10_std',
 'quality_11_std',
 'quality_12_std',
 '04.22.1684_std',
 '04.33.1125_std',
 '04.73.2237_std',
 '05.15.2138_std',
 '04.16.3439_std',
 '04.22.1750_std',
 '05.66.3237_std',
 '03.11.1167_std',
 '04.73.2571_std',
 '04.82.1684_

In [ ]:
quality_df= pd.read_csv(defaultpath+'/dataset/train_quality_data.csv',encoding='utf-8',dtype=np.str)
test_quality_df= pd.read_csv(defaultpath+'/dataset/test_quality_data.csv',encoding='utf-8',dtype=np.str)
quality_df.drop_duplicates(inplace=True)
quality_df.drop(['quality_3','quality_4'],axis=1,inplace=True)
for col in quality_df.columns:
    try : quality_df[col] = quality_df[col].str.replace(',','')
    except: continue
quality_df.iloc[:,3:] = quality_df.iloc[:,3:].astype(np.float)
quality_df['user_id'] = quality_df['user_id'].astype(np.int32)
quality_df['month']=quality_df['time'].apply(lambda x:x[4:6])
quality_df['day']=quality_df['time'].apply(lambda x:x[6:8])
quality_df['hour']=quality_df['time'].apply(lambda x:x[8:10])
quality_df.fillna(0,inplace=True)
quality_df = pd.concat([quality_df,pd.get_dummies(quality_df['fwver']).loc[:,list(set(quality_df['fwver'].values)& set(test_quality_df['fwver'].values))]],axis=1).drop('fwver',axis=1)
l = []
for key in ['_q_mean','_q_std','_q_skew','_q_sum','_q_kurt']:
    for col in quality_df.drop(['month','day','hour'],axis=1).columns[2:]:
        l = l+[str(col)+key]
quality_arr = pd.DataFrame(np.zeros((15000,165)),columns=l)
for i in tqdm(range(10000,25000,1)):
    temp = quality_df[quality_df['user_id']==i]
    if len(temp) ==0: continue
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_q = temp.groupby(['month','day','hour']).sum().drop(['user_id'],axis=1).reset_index().drop(['month','day','hour'],axis=1)
    mean = temp_q.mean().values
    std = temp_q.std().values
    skew = temp_q.skew()
    sum = temp_q.sum()
    kurt = temp_q.kurt()
    quality_arr.loc[i-10000,:] = np.hstack([mean,std,skew,sum,kurt])
problem_df= pd.read_csv(defaultpath+'/dataset/train_problem_data.csv',encoding='utf-8',dtype=np.str)
problem_ppdf = pd.merge(problem_df.astype(np.int32).drop_duplicates(),pd.DataFrame(range(10000,25000),columns=['user_id']),how='outer',on='user_id')
problem_ppdf['time'][problem_ppdf['time'].notna()] = 1
problem_ppdf.fillna(0,inplace=True)
problem_ppdf = problem_ppdf.astype(np.int32)
problem_ppdf.sort_values('user_id',inplace=True)
problem_ppdf.drop_duplicates(inplace=True)
quality_arr = pd.concat([problem_ppdf.reset_index(drop=True),quality_arr],axis=1)
quality_arr.fillna(0,inplace=True)
quality_arr.to_pickle(defaultpath+'/dataset/pp_quality.pkl')

In [ ]:
quality_arr

,user_id,time,quality_0_q_mean,quality_1_q_mean,quality_2_q_mean,quality_5_q_mean,quality_6_q_mean,quality_7_q_mean,quality_8_q_mean,quality_9_q_mean,quality_10_q_mean,quality_11_q_mean,quality_12_q_mean,04.22.1684_q_mean,04.33.1125_q_mean,04.73.2237_q_mean,05.15.2138_q_mean,04.16.3439_q_mean,04.22.1750_q_mean,05.66.3237_q_mean,03.11.1167_q_mean,04.73.2571_q_mean,04.82.1684_q_mean,03.11.1149_q_mean,05.66.3571_q_mean,04.16.3571_q_mean,09.17.1431_q_mean,04.22.1778_q_mean,04.33.1185_q_mean,05.15.2120_q_mean,04.33.1149_q_mean,04.82.1778_q_mean,04.16.3553_q_mean,04.33.1261_q_mean,05.15.2114_q_mean,quality_0_q_std,quality_1_q_std,quality_2_q_std,quality_5_q_std,quality_6_q_std,...,04.33.1185_q_sum,05.15.2120_q_sum,04.33.1149_q_sum,04.82.1778_q_sum,04.16.3553_q_sum,04.33.1261_q_sum,05.15.2114_q_sum,quality_0_q_kurt,quality_1_q_kurt,quality_2_q_kurt,quality_5_q_kurt,quality_6_q_kurt,quality_7_q_kurt,quality_8_q_kurt,quality_9_q_kurt,quality_10_q_kurt,quality_11_q_kurt,quality_12_q_kurt,04.22.1684_q_kurt,04.33.1125_q_kurt,04.73.2237_q_kurt,05.15.2138_q_kurt,04.16.3439_q_kurt,04.22.1750_q_kurt,05.66.3237_q_kurt,03.11.1167_q_kurt,04.73.2571_q_kurt,04.82.1684_q_kurt,03.11.1149_q_kurt,05.66.3571_q_kurt,04.16.3571_q_kurt,09.17.1431_q_kurt,04.22.1778_q_kurt,04.33.1185_q_kurt,05.15.2120_q_kurt,04.33.1149_q_kurt,04.82.1778_q_kurt,04.16.3553_q_kurt,04.33.1261_q_kurt,05.15.2114_q_kurt
0,10000,0,0.000000,0.000000,0.000000,0.324324,0.000000,0.000000,0.0,0.000000,0.648649,0.000000,0.0,0.0,0.0,0.0,0.108108,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.453999,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,23.564566,0.000000,0.00000,0.0,0.0,23.564566,0.000000,0.0,0.0,0.0,0.0,15.766531,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,10001,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,10002,0,0.001621,-0.001621,0.000000,0.048622,0.072934,0.155592,0.0,0.003241,0.153971,-0.001621,0.0,0.0,0.0,0.0,0.035656,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.090079,0.040258,0.056980,0.504109,1.773597,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,419.111730,617.000000,308.000000,154.402122,613.807953,606.91334,0.0,617.0,223.604181,617.000000,0.0,0.0,0.0,0.0,100.530194,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,10003,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,10004,1,-0.026316,-0.026316,-0.026316,0.078947,2.263158,4.578947,0.0,0.000000,0.368421,-0.026316,0.0,0.0,0.0,0.0,0.000000,0.0,0.157895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.162221,0.162221,0.162221,0.358795,14.118579,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.000000,38.000000,38.000000,24.248039,37.989147,38.00000,0.0,0.0,35.798064,38.000000,0.0,0.0,0.0,0.0,0.000000,0.0,24.248039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
quality_df= pd.read_csv(defaultpath+'/dataset/train_quality_data.csv',encoding='utf-8',dtype=np.str)
test_quality_df= pd.read_csv(defaultpath+'/dataset/test_quality_data.csv',encoding='utf-8',dtype=np.str)
test_quality_df.drop_duplicates(inplace=True)
test_quality_df.drop(['quality_3','quality_4'],axis=1,inplace=True)
for col in test_quality_df.columns:
    try : test_quality_df[col] = test_quality_df[col].str.replace(',','')
    except: continue
test_quality_df.iloc[:,3:] = test_quality_df.iloc[:,3:].astype(np.float)
test_quality_df['user_id'] = test_quality_df['user_id'].astype(np.int32)
test_quality_df['month']=test_quality_df['time'].apply(lambda x:x[4:6])
test_quality_df['day']=test_quality_df['time'].apply(lambda x:x[6:8])
test_quality_df['hour']=test_quality_df['time'].apply(lambda x:x[8:10])
test_quality_df.fillna(0,inplace=True)
test_quality_df = pd.concat([test_quality_df,pd.get_dummies(test_quality_df['fwver']).loc[:,list(set(quality_df['fwver'].values)& set(test_quality_df['fwver'].values))]],axis=1).drop('fwver',axis=1)
l = []
for key in ['_q_mean','_q_std','_q_skew','_q_sum','_q_kurt']:
    for col in test_quality_df.drop(['month','day','hour'],axis=1).columns[2:]:
        l = l+[str(col)+key]
quality_arr = pd.DataFrame(np.zeros((15000,165)),columns=l)
for i in tqdm(range(30000,44999,1)):
    temp = test_quality_df[test_quality_df['user_id']==i]
    if len(temp) ==0: continue
    m = temp['time'].min()
    M = temp['time'].max()
    b = pd.DataFrame(pd.date_range(start=f"{m[:4]}-{m[4:6]}-{m[6:8]} {m[8:10]}:00:00",end=f"{M[:4]}-{M[4:6]}-{M[6:8]} {M[8:10]}:00:00",freq='h'),columns=['time'])
    #b = pd.DataFrame(pd.date_range(start="2020-10-31 23:00",end="2020-12-14 12:00",freq='h'),columns=['time'])
    b['time'] = b['time'].apply(lambda x: str(x).replace('-','').replace(' ','').replace(':',''))
    b['month']=b['time'].apply(lambda x:x[4:6])
    b['day']=b['time'].apply(lambda x:x[6:8])
    b['hour']=b['time'].apply(lambda x:x[8:10])
    b['user_id'] = i
    b['isreal']='not'
    b = pd.concat([b,temp])
    b.drop_duplicates(subset=['month','day','hour'],inplace=True)
    b = b[b['isreal']=='not']
    temp = pd.concat([temp,b]).fillna(0)
    temp_q = temp.groupby(['month','day','hour']).sum().drop(['user_id'],axis=1).reset_index().drop(['month','day','hour'],axis=1)
    mean = temp_q.mean().values
    std = temp_q.std().values
    skew = temp_q.skew()
    sum = temp_q.sum()
    kurt = temp_q.kurt()
    quality_arr.loc[i-30000,:] = np.hstack([mean,std,skew,sum,kurt])
quality_arr.to_pickle(defaultpath+'/dataset/pp_test_quality.pkl')

In [ ]:
quality_arr

,quality_0_q_mean,quality_1_q_mean,quality_2_q_mean,quality_5_q_mean,quality_6_q_mean,quality_7_q_mean,quality_8_q_mean,quality_9_q_mean,quality_10_q_mean,quality_11_q_mean,quality_12_q_mean,04.22.1684_q_mean,04.33.1125_q_mean,04.73.2237_q_mean,05.15.2138_q_mean,04.16.3439_q_mean,04.22.1750_q_mean,05.66.3237_q_mean,03.11.1167_q_mean,04.73.2571_q_mean,04.82.1684_q_mean,03.11.1149_q_mean,05.66.3571_q_mean,04.16.3571_q_mean,09.17.1431_q_mean,04.22.1778_q_mean,04.33.1185_q_mean,05.15.2120_q_mean,04.33.1149_q_mean,04.82.1778_q_mean,04.16.3553_q_mean,04.33.1261_q_mean,05.15.2114_q_mean,quality_0_q_std,quality_1_q_std,quality_2_q_std,quality_5_q_std,quality_6_q_std,quality_7_q_std,quality_8_q_std,...,04.33.1185_q_sum,05.15.2120_q_sum,04.33.1149_q_sum,04.82.1778_q_sum,04.16.3553_q_sum,04.33.1261_q_sum,05.15.2114_q_sum,quality_0_q_kurt,quality_1_q_kurt,quality_2_q_kurt,quality_5_q_kurt,quality_6_q_kurt,quality_7_q_kurt,quality_8_q_kurt,quality_9_q_kurt,quality_10_q_kurt,quality_11_q_kurt,quality_12_q_kurt,04.22.1684_q_kurt,04.33.1125_q_kurt,04.73.2237_q_kurt,05.15.2138_q_kurt,04.16.3439_q_kurt,04.22.1750_q_kurt,05.66.3237_q_kurt,03.11.1167_q_kurt,04.73.2571_q_kurt,04.82.1684_q_kurt,03.11.1149_q_kurt,05.66.3571_q_kurt,04.16.3571_q_kurt,09.17.1431_q_kurt,04.22.1778_q_kurt,04.33.1185_q_kurt,05.15.2120_q_kurt,04.33.1149_q_kurt,04.82.1778_q_kurt,04.16.3553_q_kurt,04.33.1261_q_kurt,05.15.2114_q_kurt
0,0.000000,0.000000,0.000000,2.000000,5.000000,10.000000,0.0,0.0,4.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,2.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.004444,-0.004444,-0.004444,11.382222,-0.004444,0.000000,0.0,0.0,45.533333,-0.004444,0.0,0.0,0.0,0.0,0.044444,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.066667,0.066667,0.066667,170.398681,0.066667,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225.000000,225.000000,225.000000,224.999096,225.000000,0.000000,0.0,0.0,224.999023,225.000000,0.0,0.0,0.0,0.0,78.168514,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.007092,-0.007092,-0.007092,0.307329,-0.007092,0.000000,0.0,0.0,0.907801,-0.007092,0.0,0.0,0.0,0.0,0.000000,0.0,0.061466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.084015,0.084015,0.084015,3.056361,0.084015,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,137.642903,137.642903,137.642903,185.575873,137.642903,0.000000,0.0,0.0,185.448484,137.642903,0.0,0.0,0.0,0.0,0.000000,0.0,43.075342,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.041475,1.013825,2.580645,0.0,0.0,0.138249,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.027650,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.032258,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.351101,8.744035,22.433734,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,98.401228,80.550822,83.547692,0.0,0.0,149.738557,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,68.953447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.706055,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.008264,-0.008264,-0.008264,0.008264,0.264463,0.743802,0.0,0.0,0.033058,-0.008264,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.016529,0.024793,0.0,0.090909,0.090909,0.090909,0.203955,2.238931,6.733658,0.0,...,0.0,0.0,0.0,0.0,2.0,3.0,0.0,121.000000,121.000000,121.000000,81.830621,92.106141,107.156907,0.0,0.0,121.000000,121.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,121.0,121.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.